In [1]:
import django_initializer
from telemetry.fast_lap_analyzer import FastLapAnalyzer
from telemetry.racing_stats import RacingStats
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
import numpy as np

from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()
fast_lap_analyzer = FastLapAnalyzer()
racing_stats = RacingStats()

2023-06-21 18:08:30,907 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [53]:
kwargs = {
    "game": "iRacing",
    # "track": "oschersleben gp",
    "track": "aragon gp",
    "car": "Ferrari 488 GT3 Evo 2020",
    "valid": True,
}
laps = racing_stats.laps(**kwargs)
laps = laps[:10]
fast_lap_analyzer.laps = laps
data_frames, laps_with_telemetry = fast_lap_analyzer.fetch_lap_telemetry()

2023-06-21 20:58:27,660 INFO Processing iRacing - aragon gp - Ferrari 488 GT3 Evo 2020
2023-06-21 20:58:27,661 INFO   track.id 1252 car.id 9
2023-06-21 20:58:27,662 INFO   session 1687280545 lap.id 150733 number 4
2023-06-21 20:58:27,663 INFO   length 5289 time 120.5972 valid True
2023-06-21 20:58:27,664 INFO   start 2023-06-20 19:08:23.283958+00:00 end 2023-06-20 19:10:23.911187+00:00
2023-06-21 20:58:28,271 INFO Processing iRacing - aragon gp - Ferrari 488 GT3 Evo 2020
2023-06-21 20:58:28,271 INFO   track.id 1252 car.id 9
2023-06-21 20:58:28,272 INFO   session 1687366956 lap.id 153111 number 4
2023-06-21 20:58:28,272 INFO   length 5289 time 120.8618 valid True
2023-06-21 20:58:28,272 INFO   start 2023-06-21 19:08:32.603764+00:00 end 2023-06-21 19:10:33.458762+00:00
2023-06-21 20:58:28,283 ERROR No data found for 1687366956 lap 4
2023-06-21 20:58:28,284 INFO No data found for lap
2023-06-21 20:58:28,301 INFO Processing iRacing - aragon gp - Ferrari 488 GT3 Evo 2020
2023-06-21 20:58:28

In [54]:
sector_start_end = fast_lap_analyzer.extract_sectors(data_frames)
sector_start_end

[{'start': 241, 'end': 775, 'length': 534},
 {'start': 776, 'end': 1052, 'length': 276},
 {'start': 1053, 'end': 1553, 'length': 500},
 {'start': 1554, 'end': 1922, 'length': 368},
 {'start': 1923, 'end': 2062, 'length': 139},
 {'start': 2063, 'end': 2845, 'length': 782},
 {'start': 2846, 'end': 3299, 'length': 453},
 {'start': 3300, 'end': 3451, 'length': 151},
 {'start': 3452, 'end': 4548, 'length': 1096},
 {'start': 4549, 'end': 240, 'length': 980}]

In [56]:
for i in range(len(sector_start_end)):
    # display(sector_start_end[i])
    start = sector_start_end[i]["start"]
    end = sector_start_end[i]["end"]
    sector = fast_lap_analyzer.fastest_sector(data_frames, start, end)

    segment = fast_lap_analyzer.extract_segment(sector)
    segment.start = start
    segment.end = end
    segment.turn = i + 1

    if start > end:
        # add track_length to all distances that are less than start
        sector["DistanceRoundTrack"] = sector["DistanceRoundTrack"].apply(
            lambda x: x + data_frames[0]["DistanceRoundTrack"].max() if x < start else x
        )

    fig = lap_fig(sector, columns=["Throttle", "Brake"])
    brake_features = segment.brake_features()
    if brake_features:
        fig_add_features(fig, segment.brake_features())
    throttle_features = segment.throttle_features()
    if throttle_features:
        fig_add_features(fig, segment.throttle_features(), color="green")
    fig.show()

In [10]:
# new_dataframes = [df[df['Throttle'] == 1] for df in lap_df]
# result_df = pd.concat(new_dataframes)
# # sort by DistanceRoundTrack
# result_df = result_df.sort_values(by=['DistanceRoundTrack'])
# result_df = result_df.reset_index(drop=True)

# result_df
# fig = lap_fig(result_df, full_range=True, mode='markers')
# fig.show()

df_max = lap_df[1][["Throttle"]]
df_max = lap_df[-1]

# fig = lap_fig(df_max, full_range=True)
# fig.show()

for df in lap_df[0:-1]:
    # fig = lap_fig(df, full_range=True)
    # fig.show()

    df_max_throttle = df_max[["Throttle"]].combine(df[["Throttle"]], np.maximum)
    df_max["Throttle"] = df_max_throttle
    # fig = lap_fig(df_max, full_range=True)
    # fig.show()

# display(df_max)
# fig = lap_fig(df_max, full_range=True)
# fig.show()

sectors = analyzer.split_sectors(df_max)
sector_start_end = analyzer.extract_sector_start_end(sectors)
sector_start_end

[{'start': 318, 'end': 879, 'length': 561},
 {'start': 880, 'end': 1349, 'length': 469},
 {'start': 1350, 'end': 1623, 'length': 273},
 {'start': 1624, 'end': 1962, 'length': 338},
 {'start': 1963, 'end': 2614, 'length': 651},
 {'start': 2615, 'end': 3013, 'length': 398},
 {'start': 3014, 'end': 3199, 'length': 185},
 {'start': 3200, 'end': 317, 'length': 725}]

In [24]:
i = 3
for j in range(len(lap_df)):
    print(f"length: {sector_start_end[i]['end'] - sector_start_end[i]['start']}")
    track_df = lap_df[j]
    section_df = analyzer.section_df(track_df, sector_start_end[i]["start"], sector_start_end[i]["end"])
    section_time = section_df.iloc[-1]["Time"] - section_df.iloc[0]["Time"]
    print(f"time: {section_time / 10_000_000_00}")

length: 338
time: 10.099425024
length: 338
time: 10.07880704
length: 338
time: 10.10236416
length: 338
time: 10.0097728
length: 338
time: 9.95799296
length: 338
time: 9.975236096
length: 338
time: 10.005091072
length: 338
time: 10.062951936
length: 338
time: 10.072158976
length: 338
time: 10.113069056


In [ ]:
brake_feature_args = {
    "brake_threshold": 0.1,
}
throttle_features_args = {
    # "throttle_threshold": 0.98,
}
sector_dfs = []

track_df = df_max
for i in range(len(sector_start_end)):
    start = sector_start_end[i]["start"]
    end = sector_start_end[i]["end"]
    sector = analyzer.section_df(track_df, sector_start_end[i]["start"], sector_start_end[i]["end"])
    if start > end:
        # add track_length to all distances that are less than start
        sector["DistanceRoundTrack"] = sector["DistanceRoundTrack"].apply(
            lambda x: x + track_df["DistanceRoundTrack"].max() if x < start else x
        )
    sector_dfs.append(sector)
    fig = lap_fig(sector, columns=["Throttle", "Brake"])
    # fig = lap_fig(sector, columns=["Throttle", "Brake", "SteeringAngle"])
    # lap_fig(sector, columns=["SteeringAngle"], fig=fig)

    throttle_or_brake = analyzer.sector_type(sector)

    # if throttle_or_brake == "brake":
    #     continue
    # if i != 2:
    #     continue
    # fig.show()
    title = f"{i} - {throttle_or_brake}<br>"

    brake_features = analyzer.brake_features(sector, **brake_feature_args)
    title += f"brake: {brake_features}<br>"
    if brake_features:  # and throttle_or_brake == "brake":
        fig_add_features(fig, brake_features)

    throttle_features = analyzer.throttle_features(sector, **throttle_features_args)
    if throttle_features:  # and throttle_or_brake == "throttle":
        fig_add_features(fig, throttle_features, color="green")
    title += f"throttle: {throttle_features}"

    # fig.update_layout(title=dict(text=title, font=dict(size=8)))

    fig.show()

In [ ]:
# print(analysis[0]['segments'][0].start)